# Enumerating polyominoes
Adapted from https:#rosettacode.org/wiki/Free_polyominoes_enumeration#Python

In [1]:
from itertools import groupby, chain
from operator import itemgetter
from sys import argv
from array import array
import numpy as np
import json
import utils

def concat_map(func, it):
    return list(chain.from_iterable(map(func, it)))

def minima(poly):
    """Finds the min x and y coordiate of a Polyomino."""
    return (min(pt[0] for pt in poly), min(pt[1] for pt in poly))

def translate_to_origin(poly):
    (minx, miny) = minima(poly)
    return [(x - minx, y - miny) for (x, y) in poly]

rotate90   = lambda x_y:    (x_y[1], -x_y[0])
rotate180  = lambda x_y1: (-x_y1[0], -x_y1[1])
rotate270  = lambda x_y2: (-x_y2[1],  x_y2[0])
reflect    = lambda x_y3: (-x_y3[0],  x_y3[1])

def rotations_and_reflections(poly):
    """All the plane symmetries of a rectangular region."""
    return (poly,
            list(map(rotate90, poly)),
            list(map(rotate180, poly)),
            list(map(rotate270, poly)),
            list(map(reflect, poly)),
            [reflect(rotate90(pt)) for pt in poly],
            [reflect(rotate180(pt)) for pt in poly],
            [reflect(rotate270(pt)) for pt in poly])

def canonical(poly):
    return min(sorted(translate_to_origin(pl)) for pl in rotations_and_reflections(poly))

def unique(lst):
    lst.sort()
    return list(map(next, map(itemgetter(1), groupby(lst))))

# All four points in Von Neumann neighborhood.
contiguous = lambda x_y4: [(x_y4[0] - 1, x_y4[1]), (x_y4[0] + 1, x_y4[1]), (x_y4[0], x_y4[1] - 1), (x_y4[0], x_y4[1] + 1)]

def new_points(poly):
    """Finds all distinct points that can be added to a Polyomino."""
    return unique([pt for pt in concat_map(contiguous, poly) if pt not in poly])

def new_polys(poly):
    return unique([canonical(poly + [pt]) for pt in new_points(poly)])

monomino = [(0, 0)]
monominoes = [monomino]

def rank(n):
    """Generates polyominoes of rank n recursively."""
    assert n >= 0
    if n == 0: return []
    if n == 1: return monominoes
    return unique(concat_map(new_polys, rank(n - 1)))

def getCoords(n):
    return [[np.array([c[0], c[1], 0]) for c in p] for p in rank(n)]

In [2]:
def topFromCoords(coords):
    neigbourDirs = utils.getRuleOrder(3)

    bindings = []
    donePairs = []  # Keep track so that only one bond per pair is saved

    # For each position
    for i, current in enumerate(coords):
        # Enumerate von Neumann neighborhood
        for dPi, dP in enumerate(neigbourDirs):
            neigbourPos = current + dP
            # Check if current neighbor is among the positions
            for j, other in enumerate(coords):
                if np.array_equal(neigbourPos, other):
                    if str(sorted([i,j])) not in donePairs:
                        bindings.append([
                            # Particle {} patch {}
                            i, dPi,
                            # with Particle {} patch {}
                            j, dPi + (1 if dPi % 2 == 0 else -1)
                        ])
                        #print("Particle {} ({}) patch {} with particle {} ({}) patch {}".format(i, current, dPi,j, other, dPi + (1 if dPi % 2 == 0 else -1)))
                        donePairs.append(str(sorted([i,j])))
    return bindings

In [3]:
def generateFiles(n, directory='.'):
    for i, p in enumerate(getCoords(n)):
        with open("{}/{}-mer_{}.json".format(directory, n, i), "w") as outfile:
            json.dump({
                'nDim': 3,
                'bindings': topFromCoords(p),
                'torsion': True,
                'stopAtFirst': True
            }, outfile, indent=4)

In [4]:
generateFiles(8, '../shapes/8-mer_polyominoes')